<a href="https://colab.research.google.com/github/minsugaa1993/SQL/blob/main/PROJECT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import sqlite3
from google.colab import files

# 1. Upload dataset manual

In [2]:
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Get the uploaded file name


Saving Nashville_housing_data_2013_2016.csv to Nashville_housing_data_2013_2016.csv


# 2. Load dataset

In [3]:
df = pd.read_csv(file_path)

# 3. Rename columns to remove spaces and special characters

In [4]:
df.columns = df.columns.str.replace('[^A-Za-z0-9_]', '', regex=True)

# 4. Create SQLite database in memory

In [5]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# 5. Save DataFrame to SQL database

In [6]:
df.to_sql('nashville_housing', conn, index=False, if_exists='replace')

56636

# 6. Check first 5 rows

In [7]:
query = "SELECT * FROM nashville_housing LIMIT 5;"
print(pd.read_sql(query, conn))

   Unnamed01  Unnamed0          ParcelID            LandUse  \
0          0         0  105 03 0D 008.00  RESIDENTIAL CONDO   
1          1         1   105 11 0 080.00      SINGLE FAMILY   
2          2         2   118 03 0 130.00      SINGLE FAMILY   
3          3         3   119 01 0 479.00      SINGLE FAMILY   
4          4         4   119 05 0 186.00      SINGLE FAMILY   

     PropertyAddress SuiteCondo PropertyCity    SaleDate  SalePrice  \
0    1208  3RD AVE S          8    NASHVILLE  2013-01-24     132000   
1   1802  STEWART PL       None    NASHVILLE  2013-01-11     191500   
2  2761  ROSEDALE PL       None    NASHVILLE  2013-01-18     202000   
3  224  PEACHTREE ST       None    NASHVILLE  2013-01-18      32000   
4      316  LUTIE ST       None    NASHVILLE  2013-01-23     102000   

     LegalReference  ... BuildingValue TotalValue FinishedArea FoundationType  \
0  20130128-0008725  ...           NaN        NaN          NaN           None   
1  20130118-0006337  ...      13

# 7. Remove duplicate rows

In [8]:
query = """
DELETE FROM nashville_housing
WHERE rowid NOT IN (
    SELECT MIN(rowid) FROM nashville_housing GROUP BY ParcelID, SaleDate, SalePrice
);
"""
cursor.execute(query)
conn.commit()


# 8. Fill NULL values

In [9]:
query = """
UPDATE nashville_housing
SET PropertyCity = COALESCE(PropertyCity, 'Unknown');
"""
cursor.execute(query)
conn.commit()

# 9. Standardize date format (convert SaleDate to YYYY-MM-DD)

In [10]:
query = """
UPDATE nashville_housing
SET SaleDate = substr(SaleDate, 7, 4) || '-' || substr(SaleDate, 1, 2) || '-' || substr(SaleDate, 4, 2);
"""
cursor.execute(query)
conn.commit()

# 10. Check cleaned data

In [11]:
query = "SELECT * FROM nashville_housing;"
cleaned_df = pd.read_sql(query, conn)
print(cleaned_df.head())

   Unnamed01  Unnamed0          ParcelID            LandUse  \
0          0         0  105 03 0D 008.00  RESIDENTIAL CONDO   
1          1         1   105 11 0 080.00      SINGLE FAMILY   
2          2         2   118 03 0 130.00      SINGLE FAMILY   
3          3         3   119 01 0 479.00      SINGLE FAMILY   
4          4         4   119 05 0 186.00      SINGLE FAMILY   

     PropertyAddress SuiteCondo PropertyCity    SaleDate  SalePrice  \
0    1208  3RD AVE S          8    NASHVILLE  1-24-20-3-     132000   
1   1802  STEWART PL       None    NASHVILLE  1-11-20-3-     191500   
2  2761  ROSEDALE PL       None    NASHVILLE  1-18-20-3-     202000   
3  224  PEACHTREE ST       None    NASHVILLE  1-18-20-3-      32000   
4      316  LUTIE ST       None    NASHVILLE  1-23-20-3-     102000   

     LegalReference  ... BuildingValue TotalValue FinishedArea FoundationType  \
0  20130128-0008725  ...           NaN        NaN          NaN           None   
1  20130118-0006337  ...      13

# 11. Save cleaned data to CSV

In [12]:
cleaned_file_path = 'Nashville_housing_cleaned.csv'
cleaned_df.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved to {cleaned_file_path}")

Cleaned dataset saved to Nashville_housing_cleaned.csv


# 12. Download cleaned data

In [13]:
files.download(cleaned_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Close the database connection

In [14]:
conn.close()